# Deep Q-Learning

For this assignment we will implement the Deep Q-Learning algorithm with Experience Replay as described in breakthrough paper __"Playing Atari with Deep Reinforcement Learning"__. We will be testing the Q-Learning algorithm in the __"Highway Environment"__. In this environment our Q-Learning agent will operate a green car on a highway with other cars. The agent is penalized for hitting other cars and rewarded for driving faster.

In [ ]:
# Download needed libraries
# If running outside of Google Colab, these lines may not be needed
# Install environment and libraries to render environment
!sudo apt-get update
!pip install git+https://github.com/DLR-RM/stable-baselines3 &> /dev/null
!pip install highway-env &> /dev/null
!pip install xvfbwrapper &> /dev/null
!apt-get install -y xvfb python-opengl x11-utils ffmpeg build-essential python-dev swig python-pygame &> /dev/null
!pip install PyVirtualDisplay &> /dev/null

# install dependencies
!apt install chromium-browser xvfb

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [631 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security/multiverse amd64 Packages [44.0 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,204 kB]
Hit:9 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,011 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:13 https://pp

In [ ]:
# #############################################################################
# # TODO: Edit the path to your assignment folder                             #
# #############################################################################
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir("/content/gdrive/My Drive/CIVE 6358/Assignment 5")
# #############################################################################

In [ ]:
#importing needed libraries
#DO NOT ADD OR MODIFY THIS CELL
from IPython import display as ipythondisplay
from pyvirtualdisplay import Display
from gym.wrappers import record_video
from pathlib import Path
import base64
import gym
import highway_env
from matplotlib import pyplot as plt
from tqdm import tqdm
import os
import gymnasium as gym
%matplotlib inline


In the following cells you will be shown the observations from the environment that will be fed into our Q-Learning agent. We give a stack of 4 images to the agent. By passing stacks of images, we are able to give the agent information about the velocity of the agent and other cars in the environment.
<br> __No modification of this cell is needed__

In [ ]:
#DO NOT ADD OR MODIFY THIS CELL
env=gym.make('highway-fast-v0', render_mode='rgb_array')
config = {
      "observation": {
          "type": "GrayscaleObservation",
          "observation_shape": (128, 64),
          "stack_size": 4,
          "weights": [0.2989, 0.5870, 0.1140],  # weights for RGB conversion
          "scaling": 1.75,
      },
      "policy_frequency": 2
  }
env.configure(config)
env.reset()
for _ in range(4):
    obs,_,_,_,_ = env.step(env.action_type.actions_indexes["FASTER"])

    _, axes = plt.subplots(ncols=4, figsize=(12, 5))
    for i, ax in enumerate(axes.flat):
        ax.imshow(obs[i, ...].T, cmap=plt.get_cmap('gray'))
plt.show()


**What does this cell do?**

This cell will show an agent taking __random actions__ in the environment. The green car is the agent.

__By default, the cell is commented out.__

**********************************************
**Optional Instructions for Running the Cell**

1. When running this cell for the first time, execute it to generate a sample video where the agent takes random actions.

2. After running the cell and viewing the video, please comment out (disable) this cell.

3. Restart the runtime by selecting "Runtime" -> "Restart runtime" from the top menu.

4. Once the runtime is restarted, rerun the notebook starting from the beginning.


**Why Is This Important?**

Rendering videos consumes memory, and in the unpaid version of Colab, rendering both the sample and final videos in a single runtime session can lead to a crash due to memory limitations. To prevent this, it's important to comment out this cell after viewing the sample video, restart the runtime, and then rerun the code. This ensures that enough memory is available for rendering the final video.




In [ ]:
# import gymnasium as gym
# #DO NOT ADD OR MODIFY THIS CELL
# def record_videos(env, path="videos"):
#     wrapped = record_video.RecordVideo(env, path, force=True, video_callable=lambda episode: True)
#     # Capture intermediate frames
#     env.unwrapped.set_record_video_wrapper(wrapped)

#     return wrapped

# def show_videos(path="videos/random_actions"):
#     html = []
#     for mp4 in Path(path).glob("*.mp4"):
#         video_b64 = base64.b64encode(mp4.read_bytes())
#         html.append('''<video alt="{}" autoplay
#                       loop controls style="height: 400px;">
#                       <source src="data:video/mp4;base64,{}" type="video/mp4" />
#                  </video>'''.format(mp4, video_b64.decode('ascii')))
#     ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))
# from tqdm.notebook import trange
# env = gym.make("highway-fast-v0", render_mode = 'rgb_array')
# env.configure({
#     "observation": {
#         "type": "GrayscaleObservation",
#         "observation_shape": (128, 64),
#         "stack_size": 4,
#         "weights": [0.2989, 0.5870, 0.1140],  # weights for RGB conversion
#         "scaling": 1.75,
#     },
# })

# from pyvirtualdisplay import Display
# display = Display(visible=0, size=(400, 300))
# display.start()
# from gym.wrappers import RecordVideo

# def record_videos(env, path="videos/random_actions"):
#     wrapped = RecordVideo(env, path)

#     # Capture intermediate frames
#     env.unwrapped.set_record_video_wrapper(wrapped)

#     return wrapped


# env = record_videos(env)
# for episode in trange(3, desc="Test episodes"):
#     obs, done = env.reset()[0], False
#     while not done:
#       action = env.action_space.sample()
#       # print(action)
#       print(env.step(int(action)))
#       obs, reward, done, info = env.step(int(action))
#       #obs, reward, done, info = env.step(env.action_type.actions_indexes["FASTER"])
# env.close()
# show_videos()
# display.stop()

In this cell we define a small nueral network that we will use to aproximate the Q-Function.
<br> __No modification of this cell is needed__

In [ ]:
#DO NOT ADD OR MODIFY THIS CELL
import torch
from torch import nn
class Network(torch.nn.Module):
  def __init__(self):
      super(Network, self).__init__()
      n_input_channels = 4
      self.cnn = nn.Sequential(
          nn.Conv2d(n_input_channels, 32, kernel_size=8, stride=4, padding=0),
          nn.ReLU(),
          nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=0),
          nn.ReLU(),
          nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=0),
          nn.ReLU(),
          nn.Flatten(),
          nn.Linear(3072,3072),
          nn.ReLU(),
          nn.Linear(3072,3072),
          nn.ReLU(),
          nn.Linear(3072,5),
      )
  def forward(self,x):
    return self.cnn(x)


Experience replay implementation
<br> __No modification of this cell is needed__

In [ ]:
import collections
import numpy as np

#datastructure for 1 experience
Experience = collections.namedtuple("Experience", field_names=("state","action", "reward", "done", "new_state"))

#store and sample multiple experiences of type Experience
class ExperienceReplay:
  def __init__(self, capacity):
      self.buffer = collections.deque(maxlen=capacity)

  def __len__(self):
      return len(self.buffer)

  def append(self, experience):
      self.buffer.append(experience)

  def sample(self, batch_size):
    with torch.no_grad():
      indices = np.random.choice(len(self.buffer), batch_size,replace=False)
      states=[]
      actions=[]
      rewards=[]
      dones=[]
      next_states=[]
      for idx in indices:
        s,a,r,d,n=self.buffer[idx]
        if len(s) == 2:
          s = s[0]
        states.append(s)
        actions.append(a)
        rewards.append(r)
        dones.append(d)
        next_states.append(n)

      # Make sure the sequences have consistent lengths along the second dimension
      states = torch.Tensor(states)
      actions = torch.Tensor(actions)
      rewards = torch.Tensor(rewards).type(torch.float64)
      dones = torch.Tensor(dones).type(torch.float64)
      next_states = torch.Tensor(next_states)

      # Now, you can return these tensors
      return states, actions, rewards, dones, next_states


<br> __Modify only the sections indicated__

<br> A class called Agent is defined below that implements a Q-Learning agent:
<br>This class implements the following functions:
<br>fill_buffer(): Fills the buffer with a selected amount of state action pairs, the last episode during generation of state action pairs is saved
<br>rand_fill_buffer(): Fills the buffer by takeing random actions in the environment
<br>mean_reward(): calculates the mean reward achieved by the agent for a defined amount of episodes
<br>show_video(): function used to display video in notebook
<br>video(): record actions from agent and create videos
<br>load_save(): can be used to load a saved checkpoint to resume training, can be used to train a agent past the colab time limit.
You will need to implement the epsilon greed policy as well as the Q learning loss function.


# Only edit the code in the comment blocks "TODO:" is indicated. This appears in the fill_buffer() function where you will implement the epsilon greedy function and the train function where you will implement the loss function for Q-Learning.

In [ ]:
from torch._C import device
from pyvirtualdisplay import Display
from gym.wrappers import RecordVideo

class Agent:
  def __init__(self, env, Network):
    self.env=env
    self.net=Network()
    self.target_net=Network()
    self.target_net_update_freq=50 #update target net every 50 iters
    self.buffer=ExperienceReplay(15000) #Size of Experience buffer is set to 15000 state action pairs
    self.fill_buffer_size=1000 #initial fill buffer, during training this is automatically set to 1 episode worth of state action pairs
    self.initial_buffer_exp=1000 #this controls how many random actions to take in the env to initially fill the buffer, the exact amount will vary as the last episode is rolled out to completion
    self.epsilon=1.0 #epsilon is initially set to 1.0 the slowly decayed
    self.eps_decay=.9999 #we decay epsilon by using the following formula epsilon=epsilon*eps_decay , this is automatically done no need to implement
    self.eps_min=0.02 #minimum epsilon value
    self.batch_size=10 #batch size, number of experiences we feed the Q-Learning Network during trainnig

    #implementation  variables to store loss, reward, and set device
    self.device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    self.net.to(self.device)
    self.target_net.to(self.device)
    self.loss_list=[]
    self.loss_list_idx=[]
    self.reward_list=[]
    self.reward_list_idx=[]

    self.rand_fill_buffer()
    self.gamma=torch.tensor(0.99,dtype=torch.float64)
    isExist = os.path.exists("checkpoint")
    if not isExist:
      os.makedirs("checkpoint")
  #fills the replay buffer with state action pairs defined by self.fill_buffer_size
  #Epsilon greedy policy must be defined here
  def fill_buffer(self):
    steps=0
    not_full = True
    with torch.no_grad():
        while not_full:
          if steps>self.fill_buffer_size:
            not_full=False
          episode_done=False
          state=self.env.reset()[0]

          while not episode_done:
            #############################################################################
            # TODO: Select action based on epsilon greed policy.                        #
            # Set the action variable from None to the correct action.                  #
            # The following variables can be used:                                      #
            # np.random.random() This is a random number from 0 to 1                    #
            # self.epsilon This is the epsilon value                                    #
            # random_action A random action that can be taken in the environment        #
            # agent_action The action the Q learning agent would take                   #
            #############################################################################

            if np.random.random() < self.epsilon:
                action = self.env.action_space.sample()  # Random action
            else:
                state_tensor = torch.tensor(state, device=self.device, dtype=torch.float32)
                state_tensor = state_tensor.unsqueeze(0)  # Add batch dimension
                q_values = self.net(state_tensor)  # Get Q-values for each action
                _, action = torch.max(q_values, dim=1)  # Select action with max Q-value
                action = action.item()  # Get the actual action to take

            #############################################################################
            self.epsilon = max(self.epsilon*self.eps_decay, self.eps_min)
            new_state, reward, is_done, truncated, _ = self.env.step(action)
            len(new_state)
            exp = Experience(state,action,reward,is_done or truncated,new_state)
            self.buffer.append(exp)
            steps+=1
            #pbar2.update(1)
            episode_done=is_done or truncated
            state=new_state

#fill buffer with random actions defined by self.initial_buffer_exp
  def rand_fill_buffer(self):
    steps=0
    not_full = True
    print("filling buffer with random actions")
    with tqdm(total=self.initial_buffer_exp) as pbar:
      while not_full:
          if steps>self.initial_buffer_exp:
            not_full=False
          # get experience from 1 episode
          episode_done=False
          state=self.env.reset()[0]

          while not episode_done:
            #random action
            action = self.env.action_space.sample()
            new_state, reward, is_done, truncated, _ = self.env.step(action)
            #exp = Experience(state,action,reward,is_done,new_state)
            exp=[state,action,reward,is_done or truncated,new_state]
            self.buffer.append(exp)
            steps+=1
            pbar.update(1)
            #print(steps)
            episode_done=is_done or truncated
            state=new_state

#calculate mean reward for 20 episodes
  def mean_reward(self):
    #average reward selected number of episodes
    rewards=0
    episodes=20
    ep_len=1
    for ep in range(episodes):
        episode_done=False
        state=torch.tensor(self.env.reset()[0]).type(torch.float32)
        while not episode_done:
          out=torch.unsqueeze(torch.tensor(state),0)
          action=self.net(out.to(self.device))
          action=int(torch.max(action,dim=1)[1])
          new_state, reward, is_done, truncated, _ = self.env.step(action)
          ep_len+=1
          episode_done=is_done or truncated
          rewards+=reward
          state=torch.Tensor(new_state).type(torch.float32)

    print("average episode time steps: "+str(ep_len/episodes))
    return rewards/episodes

#used to load video in notebook
  def show_videos(self,path="videos/agent_actions"):
    html = []
    for mp4 in Path(path).glob("*.mp4"):
        video_b64 = base64.b64encode(mp4.read_bytes())
        html.append('''<video alt="{}" autoplay
                      loop controls style="height: 400px;">
                      <source src="data:video/mp4;base64,{}" type="video/mp4" />
                 </video>'''.format(mp4, video_b64.decode('ascii')))
    ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))

#record and display video of agent taking actions in environment
  def video(self):
    display = Display(visible=0, size=(400, 300))
    display.start()
    def record_videos(env, path="videos/agent_actions"):
        wrapped = RecordVideo(env, path)

        # Capture intermediate frames
        env.unwrapped.set_record_video_wrapper(wrapped)

        return wrapped
    env = gym.make("highway-fast-v0", render_mode='rgb_array')
    env.configure({
        "observation": {
            "type": "GrayscaleObservation",
            "observation_shape": (128, 64),
            "stack_size": 4,
            "weights": [0.2989, 0.5870, 0.1140],
            "scaling": 1.75,
        },
    })
    env = record_videos(env)
    for episode in range(3):
        state, done = env.reset()[0], False
        state=torch.tensor(state).type(torch.float32)
        while not done:
          out=torch.unsqueeze(torch.tensor(state),0)
          action=self.net(out.to(self.device))
          action=int(torch.max(action,dim=1)[1])
          state, reward, done, info = env.step(int(action))
          state=torch.Tensor(state).type(torch.float32)

    env.close()
    self.show_videos()

#load saved checkpoint to extend training past 3000
  def load_save(self,save_path):
    self.net.load_state_dict(torch.load(save_path))
    self.target_net.load_state_dict(torch.load(save_path))

#Used to train the Q-Learning Agent
#The loss function must be defined here
  def train(self,total_iterations):
    optimizer = torch.optim.Adam(self.net.parameters(),lr=5e-4)
    self.fill_buffer_size=1
    for idx in range(total_iterations):

      #fill buffer with exp using model and epsilon greedy policy
      self.fill_buffer()

      #We take a random batch from the replay buffer
      batch=self.buffer.sample(self.batch_size)
      #unpack batch


      #use these variable to calculate loss for Q function
      state,action,reward,is_done,next_state=batch

      #############################################################################
      # TODO: Calculate Q value, target Q value, and Loss for Q Learning          #
      # refer to lecture 17 and the following links:                     #
      # https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html  #
      # https://arxiv.org/pdf/1312.5602v1.pdf                                     #
      # Note: Use Mean Squared Error for Loss, DO NOT use Huber Loss              #
      # use the following variables:                                              #
      # self.net() :This is the current netowkr used to aproximate the Q function #
      # self.target_net() :This is the target network used in the Q learning loss #
      # state :This is the current state                                          #
      # action :This is the action taken in the current state                     #
      # reward :This is the reward for taking the action in the current state     #
      #         and going to the next_state                                       #
      # is_done :Boolean value of 0 if not done and 1 if done                     #
      # next_state :next state for taking action in current state                 #
      # Note: all of these variables have a batch dimension first                 #
      #############################################################################
      # IMPORTANT:                                                                #
      # Please refer to Algorithm 1 in the DQN paper to understand                #
      # how to calculate the Loss for a terminal state, when done is set to True/1#
      #############################################################################
      #compute Q value
      #give state to net and select action

      state = state.to(self.device, dtype=torch.float32)
      next_state = next_state.to(self.device, dtype=torch.float32)
      action = action.to(self.device, dtype=torch.long)
      reward = reward.to(self.device, dtype=torch.float32)
      is_done = is_done.to(self.device, dtype=torch.float32)

      current_q_values = self.net(state).gather(1, action.unsqueeze(-1)).squeeze(-1)

      #calculate expected value using target network
      #value if state is done is just the reward

      next_q_values = self.target_net(next_state).max(1)[0]
      expected_q_values = reward + self.gamma * next_q_values * (1 - is_done)
      # print('next_q_values', next_q_values, 'expected_q_values', expected_q_values)
      loss = torch.nn.functional.mse_loss(current_q_values, expected_q_values.detach())

      #############################################################################

      self.loss_list.append(loss.detach().clone().cpu() )
      self.loss_list_idx.append(idx+1)
      optimizer.zero_grad()
      loss.backward()

      torch.nn.utils.clip_grad_norm_(self.net.parameters(), 10)
      optimizer.step()

      if (idx+1)%self.target_net_update_freq==0:
        print("iteration count = "+str(idx+1))
        print("update target net")
        mean_rewards=self.mean_reward()
        self.reward_list.append(mean_rewards)
        self.reward_list_idx.append(idx+1)
        print("mean reward "+str(mean_rewards))
        #update target net here
        self.target_net.load_state_dict(self.net.state_dict())
        #save model every 500 iterations
        if (idx+1)%500==0:
          torch.save(self.net.state_dict(), "checkpoint/reward_"+str(int(mean_rewards)) +"_iteration_"+ str(idx+1) + "_model.pt" )
    return


Run the following cells to train your Q-Learning agent for 3000 iterations. A video will be saved in the videos/agent_actions folder.

In [ ]:
#We create an instance of the environment here
env = gym.make("highway-fast-v0", render_mode='rgb_array')
env.configure({
    "observation": {
        "type": "GrayscaleObservation",
        "observation_shape": (128, 64),
        "stack_size": 4,
        "weights": [0.2989, 0.5870, 0.1140],  # weights for RGB conversion
        "scaling": 1.75,
    },
})

# We will create an object from the Agent class called Q_agent
# We use this object's train function to train our Agent and display a video of the trained Agent
Q_agent=Agent(env,Network)


In [ ]:
Q_agent.train(100)

In [ ]:
Q_agent.video()

The following cell will display the reward your agent achieved during training vs the iterations it has completed


In [ ]:
plt.plot(Q_agent.reward_list_idx,Q_agent.reward_list)
plt.show()

The following cell will display the loss calculated for your agent during training vs the iterations it has completed

In [ ]:
plt.plot(Q_agent.loss_list_idx[1:],Q_agent.loss_list[1:])
plt.show()